In [1]:
%run python_functions_load_final.ipynb

In [2]:
import json
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import time

In [3]:
f = open('igs_10_16_pool_report.json') 
data = json.load(f)

In [7]:

base_pool_config = {'auto_shutdown': 10, 'min_idle_vm': 0}

auto_shutdown_grid = [1, 2, 3, 4, 5, 6, 10]
min_idle_vm_grid = [0, 1, 2, 3, 4, 5, 6, 7, 8]

if len(base_pool_config)>0:
    if base_pool_config['auto_shutdown'] not in auto_shutdown_grid:
        auto_shutdown_grid.append(base_pool_config['auto_shutdown'])
    if base_pool_config['min_idle_vm'] not in min_idle_vm_grid:
        min_idle_vm_grid.append(base_pool_config['min_idle_vm']) 

In [8]:
# creating clusters dataframe
clusters_pd = cluster_df(data)

# creating contentionTimelines dataframe
contentionTimelines_pd = contentionTimelines_df(data)

# merging two dataframes
cluster_contention_pd = contentionTimelines_pd.merge(clusters_pd, on='clusterId', how='left')

# filtering for only job clusters
cluster_contention_jc_pd=cluster_contention_pd[cluster_contention_pd.clusterType=='JOB_CLUSTER']

# creating cluster timeline dataframe
cluster_timelines_pd =  cluster_timeline_df(cluster_contention_jc_pd)

In [14]:
greedy_df = pd.DataFrame(columns=['auto_shutdown', 'min_idle_vm', 'total_idle_time',
                                  'idle_vm_count', 'cold_start_cnt', 'warm_start_cnt',])
cnt = 0
t1 = time.time()
for auto_shutdown in auto_shutdown_grid:

    # step 2
    cluster_groups = grouper(cluster_timelines_pd, auto_shutdown)


    # Adding group number to cluster_timelines_pd
    cluster_timelines_pd = cluster_timeline_update(cluster_timelines_pd, cluster_groups)


    # step 3
    total_groups_distance, cluster_groups = group_intervals(cluster_groups)

    # step 4, part B
    vm_usage_classes = vm_usage_v4(cluster_groups, cluster_timelines_pd, cluster_contention_jc_pd, auto_shutdown)


    # groups summary and stats
    groups_summary_org = groups_stats_v2(vm_usage_classes)

    # calculating total idle time in sec
    total_idle_time_org = groups_summary_org.idle_duration.sum()


    ## Part g: add effects of min_idle_vm
    for min_idle_vm in min_idle_vm_grid:
        total_idle_time, groups_summary = min_idle_vm_effect_v2(min_idle_vm, total_idle_time_org,
                                                             total_groups_distance, groups_summary_org)

        # creating greedy dataframe
        greedy_df.loc[cnt] = [None]*len(greedy_df.columns)
        greedy_df['auto_shutdown'].loc[cnt] = auto_shutdown
        greedy_df['min_idle_vm'].loc[cnt] = min_idle_vm
        greedy_df['total_idle_time'].loc[cnt] = total_idle_time
        
        greedy_df['idle_vm_count'].loc[cnt] = groups_summary.idle_vm_count.sum()
        greedy_df['cold_start_cnt'].loc[cnt] = groups_summary.cold_start_cnt.sum()
        greedy_df['warm_start_cnt'].loc[cnt] = groups_summary.warm_start_cnt.sum()
        cnt+=1


# finding optimal config       
opt_config = optimal_config(greedy_df)
t2 = time.time()
print(f'duration is {np.round(t2-t1,2)} seconds')
# calculating amount of improvements comapred to baseline usage
if len(base_pool_config)>0:
    percent_improvements = improvements(greedy_df, base_pool_config)
    print(f'The rate of improvements in using pool resources will be {percent_improvements}%')
    
print(f'The optimal suggested pool config is {opt_config}')        

duration is 5.87 seconds
The rate of improvements in using pool resources will be 42.4%
The optimal suggested pool config is {'auto_shutdown': 1, 'min_idle_vm': 0}


In [11]:
vm_usage_classes

{1: {'cold_start_cnt': 24.0,
  'warm_start_cnt': 7.0,
  'idle_duration': 23618.0,
  'idle_vm_count': 42.0},
 2: {'cold_start_cnt': 44.0,
  'warm_start_cnt': 11.0,
  'idle_duration': 25281.0,
  'idle_vm_count': 48.0},
 3: {'cold_start_cnt': 4.0,
  'warm_start_cnt': 4.0,
  'idle_duration': 4296.0,
  'idle_vm_count': 8.0},
 4: {'cold_start_cnt': 2.0,
  'warm_start_cnt': 0.0,
  'idle_duration': 1200.0,
  'idle_vm_count': 0.0},
 5: {'cold_start_cnt': 2.0,
  'warm_start_cnt': 0.0,
  'idle_duration': 1200.0,
  'idle_vm_count': 0.0},
 6: {'cold_start_cnt': 2.0,
  'warm_start_cnt': 6.0,
  'idle_duration': 3747.0,
  'idle_vm_count': 8.0},
 7: {'cold_start_cnt': 2.0,
  'warm_start_cnt': 8.0,
  'idle_duration': 4104.0,
  'idle_vm_count': 10.0},
 8: {'cold_start_cnt': 24.0,
  'warm_start_cnt': 65.0,
  'idle_duration': 29800.0,
  'idle_vm_count': 89.0},
 9: {'cold_start_cnt': 117.0,
  'warm_start_cnt': 141.0,
  'idle_duration': 100386.0,
  'idle_vm_count': 256.0}}

In [71]:
vm_pos

,index,clusterId,vmCount,startUnixTime,duration,endUnixTime,clusterName,jobId,jobName,jobRunId,group_number,vm_scale,time_diff_fw,time_diff_bw,start_availability_time,end_availability_time,idle_duration,vm_reused_cnt
0,2,0710-065105-b4vpgsf4,26.0,1688972492,PT17S,1688972509,job-724206447383105-run-51333886,724206447383105,Demand Normal Predictions - DLM,51333536,1,26.0,0.0,109.0,1688972509,1688973109,88.0,1.0
1,7,0710-065910-yuikv067,8.0,1688972532,PT1M5S,1688972597,job-16671-run-51334769,16671,Train - HWClaims,51334007,1,3.0,0.0,0.0,1688972597,1688973197,0.0,0.0
2,8,0710-065910-yuikv067,5.0,1688972597,PT6S,1688972603,job-16671-run-51334769,16671,Train - HWClaims,51334007,1,5.0,0.0,71.0,1688972603,1688973203,0.0,0.0
3,21,0710-070212-xd670eqh,1.0,1688974251,PT32M56S,1688976227,job-1042075642967192-run-51336276,1042075642967192,MergeDemandPredictions - Normals,51335414,1,1.0,0.0,3570.0,1688976227,1688976827,0.0,0.0
4,23,0710-070417-3nvs44u3,7.0,1688972807,PT1M11S,1688972878,job-9284-run-51337768,9284,HWClaims Predictions,51336955,1,7.0,0.0,-28.0,1688972878,1688973478,0.0,0.0
